In [40]:
# Note: The openai-python library support for Azure OpenAI is in preview.

import os
import openai

openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_ENDPOINT")
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# OpenAI

# Keep track of all messages
messages = []

# Set system message
messages.append(
    {
        "role": "system",
        "content": "You are an AI assistant for helping with online shopping. You generate a list of 10 products from given categories (Electronics, Toys, Apparel), with pricing ranges between $2 to $30 randomly and store it. When a user asks about products, display the list of products in CSV format. When a user wants to buy any product, ask him/her for confirmation first and then add the items to a cart array. Same items in the cart array should be grouped by adding the amount column. When the user wants to checkout, show him/her all items in the cart and ask for confirmation. If the user confirms, generate the final receipt and format it as JSON.",
    }
)


def openai_completion(prompt):
    # Append user's message
    messages.append(
        {
            "role": "user",
            "content": prompt,
        }
    )

    response = openai.ChatCompletion.create(
        engine="gpt35",
        messages=messages,
        max_tokens=4000,
        temperature=1,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
    )

    return response.choices[0].message.content

In [ ]:
# Panel Dashboard
# https://github.com/sophiamyang/panel_openai/blob/main/panel_openai.ipynb

import panel as pn

pn.extension(loading_spinner="dots", loading_color="#00aa41")

inp = pn.widgets.TextInput(value="", placeholder="Enter text here...")
button_conversation = pn.widgets.Button(name="Chat!")
convos = []  # store all panel objects in a list


def get_conversations(_):
    prompt = inp.value
    inp.value = ""
    if prompt != "":
        openai_answer = openai_completion(prompt)

        # Append AI's message
        messages.append(
            {
                "role": "assistant",
                "content": openai_answer,
            }
        )

        convos.append(pn.Row("\U0001F60A", pn.pane.Markdown(prompt, width=600)))
        convos.append(
            pn.Row(
                "\U0001F916",
                pn.pane.Markdown(
                    openai_answer, width=600, styles={"background-color": "#F6F6F6"}
                ),
            )
        )

    return pn.Column(*convos)


interactive_conversation = pn.bind(get_conversations, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=500),
)

dashboard.servable()